In [25]:
import pandas as pd 
from datetime import datetime, date, time, timedelta

df = pd.read_csv("crime.csv") 


In [26]:
df['DATE'], df['TIME'] = df['FIRST_OCCURRENCE_DATE'].str.split(' ', 1).str
df['DATE'] = pd.to_datetime(df['DATE'])
df['day_of_week'] = df['DATE'].dt.day_name()


In [27]:
df_WCC = df[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID', 'GEO_LON', 'GEO_LAT']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime'].copy()
df_WCC_am =  df_WCC.loc[df_WCC.TIME.str[-2:] == "AM" ].copy()
df_WCC_am['TIME'] = [x[:-3] for x in df_WCC_am['TIME']]
df_WCC_am['TIME'] =  pd.to_datetime(df_WCC_am['TIME'], format='%H:%M:%S')

df_WCC = df_WCC[['OFFENSE_CATEGORY_ID', 'TIME', 'day_of_week','NEIGHBORHOOD_ID', 'GEO_LON', 'GEO_LAT']]
df_WCC = df_WCC.loc[df_WCC.OFFENSE_CATEGORY_ID == 'white-collar-crime'].copy()
df_WCC_pm =  df_WCC.loc[df_WCC.TIME.str[-2:] == "PM" ].copy()
df_WCC_pm['TIME'] = [x[:-3] for x in df_WCC_pm['TIME']]
df_WCC_pm['TIME'] =  pd.to_datetime(df_WCC_pm['TIME'], format='%H:%M:%S')
df_WCC_pm['TIME'] = df_WCC_pm['TIME'] + timedelta(hours=12)

In [28]:
final_WCC = pd.concat([df_WCC_am, df_WCC_pm], ignore_index=True, sort =False)

block1 = '09/19/18 04:00:00'
block2 = '09/19/18 08:00:00'
block3 = '09/19/18 12:00:00'
block4 = '09/19/18 16:00:00'
block5 = '09/19/18 20:00:00'
block6 = '09/19/18 23:59:00'
datetime_object1 = datetime.strptime(block1, '%m/%d/%y %H:%M:%S')
datetime_object2 = datetime.strptime(block2, '%m/%d/%y %H:%M:%S')
datetime_object3 = datetime.strptime(block3, '%m/%d/%y %H:%M:%S')
datetime_object4 = datetime.strptime(block4, '%m/%d/%y %H:%M:%S')
datetime_object5 = datetime.strptime(block5, '%m/%d/%y %H:%M:%S')
datetime_object6 = datetime.strptime(block6, '%m/%d/%y %H:%M:%S')


In [29]:
final_WCC.loc[final_WCC['TIME'].dt.time <= datetime_object1.time(), 'TIME'] = datetime_object1

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object1.time())&(final_WCC['TIME'].dt.time <= datetime_object2.time()), 'TIME'] = datetime_object2

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object2.time())&(final_WCC['TIME'].dt.time <= datetime_object3.time()), 'TIME'] = datetime_object3

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object3.time())&(final_WCC['TIME'].dt.time <= datetime_object4.time()), 'TIME'] = datetime_object4

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object4.time())&(final_WCC['TIME'].dt.time <= datetime_object5.time()), 'TIME'] = datetime_object5

final_WCC.loc[(final_WCC['TIME'].dt.time > datetime_object5.time())&(final_WCC['TIME'].dt.time <= datetime_object6.time()), 'TIME'] = datetime_object6



final_WCC['TIME'] = [d.time() for d in final_WCC['TIME']]
final_WCC

,OFFENSE_CATEGORY_ID,TIME,day_of_week,NEIGHBORHOOD_ID,GEO_LON,GEO_LAT
0,white-collar-crime,04:00:00,Tuesday,dia,-104.771233,39.809242
1,white-collar-crime,12:00:00,Thursday,capitol-hill,-104.981438,39.738707
2,white-collar-crime,12:00:00,Thursday,five-points,-104.981133,39.750645
3,white-collar-crime,12:00:00,Wednesday,harvey-park,-105.029152,39.668702
4,white-collar-crime,12:00:00,Sunday,chaffee-park,-105.016657,39.784930
...,...,...,...,...,...,...
7201,white-collar-crime,16:00:00,Thursday,north-capitol-hill,-104.987050,39.744489
7202,white-collar-crime,16:00:00,Thursday,lincoln-park,-105.000068,39.740108
7203,white-collar-crime,20:00:00,Monday,dia,-104.704438,39.836504
7204,white-collar-crime,16:00:00,Thursday,lincoln-park,-105.014162,39.740439


In [35]:
WCC_csv = final_WCC[['day_of_week','NEIGHBORHOOD_ID', 'TIME']]
WCC_plot = final_WCC[['day_of_week','NEIGHBORHOOD_ID', 'TIME','GEO_LAT','GEO_LON']]

WCC_csv = WCC_csv.sample(6000)

WCC_csv.to_csv('WCC.csv')
WCC_plot.to_csv('WCC_Plot.csv')


!python apriori.py -f 'WCC.csv' -s 0.0025

item: ('Thursday', 'dia', '12:00:00') , 0.003
item: ('Wednesday', '16:00:00', 'cherry-creek') , 0.003
item: ('20:00:00', 'dia', 'Saturday') , 0.003
item: ('cbd', 'Thursday', '16:00:00') , 0.003
item: ('Wednesday', 'dia', '12:00:00') , 0.003
item: ('montbello', '12:00:00', 'Friday') , 0.003
item: ('20:00:00', 'Thursday', 'dia') , 0.003
